In [39]:
import sys, os
import numpy as np
import cv2
from shutil import copy

In [2]:
 def __loadfile(root, data_file, labels_file=None):
    labels = None
    if labels_file:
        path_to_labels = os.path.join(root, labels_file)
        with open(path_to_labels, 'rb') as f:
            labels = np.fromfile(f, dtype=np.uint8) - 1  # 0-based

    path_to_data = os.path.join(root, data_file)
    with open(path_to_data, 'rb') as f:
        # read whole file in uint8 chunks
        everything = np.fromfile(f, dtype=np.uint8)
        images = np.reshape(everything, (-1, 3, 96, 96))
        images = np.transpose(images, (0, 1, 3, 2))

    return images, labels

In [29]:
path_dataset = '/home/hyunjoon/dataset/stl-10/stl10_binary/'
path_res = '/home/hyunjoon/dataset/stl-10-images/'
if not os.path.exists(path_res):
    os.mkdir(path_res)

In [26]:
# training dataset
fn_images = 'train_X.bin'
fn_labels = 'train_y.bin'
images, labels = __loadfile(path_dataset, fn_images, fn_labels)

annots = ''
for ii, (img, label) in enumerate(zip(images, labels)):
    img = np.transpose(img, (1, 2, 0))
    fn_img = '{:05d}_{:03d}.jpg'.format(ii, label)
    cv2.imwrite(os.path.join(path_res, fn_img), img[:, :, ::-1])
    annots += '{} {}\n'.format(fn_img, label)

with open(os.path.join(path_res, 'train.txt'), 'w') as fh:
    fh.write(annots)

In [30]:
# validation dataset
fn_images = 'test_X.bin'
fn_labels = 'test_y.bin'
images, labels = __loadfile(path_dataset, fn_images, fn_labels)

annots = ''
for ii, (img, label) in enumerate(zip(images, labels)):
    img = np.transpose(img, (1, 2, 0))
    fn_img = '{:05d}_{:03d}.jpg'.format(ii+10000, label)
    cv2.imwrite(os.path.join(path_res, fn_img), img[:, :, ::-1])
    annots += '{} {}\n'.format(fn_img, label)
    
with open(os.path.join(path_res, 'validation.txt'), 'w') as fh:
    fh.write(annots)

In [32]:
def create_transforms(img, scale, aspect, trans, num_frames, mean_bgr):
    '''
    scale: (scale_begin, scale_end)
    trans: ((tx0, ty0), (tx1, ty1))
    aspect: (aspect_begin, aspect_end)
    num_frames: number of augmented images to create
    '''
    # interpolation weights
    alphas = np.arange(num_frames) / (num_frames - 1.0)
    
    img_hw = img.shape[:2]

    def _compute_roi(ss, asp, tx, ty, img_hw):
        sx = ss * np.sqrt(asp)
        sy = ss / np.sqrt(asp)
        
        ww = img_hw[1] / sx
        hh = img_hw[0] / sy

        x0 = tx + (img_hw[1] - ww)
        y0 = ty + (img_hw[0] - hh)
        #
        return [x0, y0, x0+ww, y0+hh]
    
    def _create_transform(roi, img_hw, mean_bgr):
        a = (img_hw[1]) / (roi[2] - roi[0])
        b = (img_hw[0]) / (roi[3] - roi[1])
        c = -a * (roi[0] - 0.0)
        d = -b * (roi[1] - 0.0)
        mapping = np.array([[a, 0, c],
                            [0, b, d]]).astype(np.float)
        func = lambda x: cv2.warpAffine(x, mapping,
                                        (img_hw[1], img_hw[0]),
                                        borderMode=cv2.BORDER_REPLICATE) #,
#                                         borderValue=mean_bgr)
        return func, mapping
    
    r_imgs = []
    for ii, a1 in enumerate(alphas):
        a0 = 1.0 - a1
        ss = scale[0] * a0 + scale[1] * a1
        asp = aspect[0] * a0 + aspect[1] * a1
        tx = trans[0][0] * a0 + trans[1][0] * a1
        ty = trans[0][1] * a0 + trans[1][1] * a1

        roi = _compute_roi(ss, asp, tx, ty, img_hw)
        trans_func, mapping = _create_transform(roi, img_hw, mean_bgr)
#         if ii == 0 or ii == len(alphas)-1:
#             print('{}: {}'.format(ii, mapping))
        
        r_imgs.append(trans_func(img))

    return r_imgs

In [33]:
def augment_image(img, scale_range, asp_range, trans_range, num_aug=5, num_frame=20):
    '''
    '''
    p_scale = np.random.uniform(scale_range[0], scale_range[1], num_aug)
    p_asp = np.random.uniform(asp_range[0], asp_range[1], num_aug)
    p_tx = np.random.uniform(trans_range[0][0], trans_range[0][1], num_aug)
    p_ty = np.random.uniform(trans_range[1][0], trans_range[1][1], num_aug)
    p_trans = np.stack([p_tx, p_ty], axis=1)
      
    r_imgs = []
    for ii in range(num_aug):
        scale = (p_scale[ii], p_scale[(ii+1) % num_aug])
        asp = (p_asp[ii], p_asp[(ii+1) % num_aug])
        trans = (p_trans[ii], p_trans[(ii+1) % num_aug])
        
        r_imgs.extend(create_transforms(img, scale, asp, trans, num_frame, [127, 127, 127]))
    
    return r_imgs

In [34]:
scale_range = (0.8, 1.25)
asp_range = (0.8, 1.25)
trans_range = ((-8, 8), (-8, 8))

In [46]:
# augment training dataset
path_dataset = '/home/hyunjoon/dataset/stl-10-images/'
path_res = '/home/hyunjoon/dataset/stl-10-biased/'
if not os.path.exists(path_res):
    os.mkdir(path_res)

with open(os.path.join(path_dataset, 'train.txt'), 'r') as fh:
    annots = fh.read().splitlines()
annots = [a.split(' ') for a in annots]
annots = [(a[0], int(a[1])) for a in annots]

In [47]:
k = 0
annot_str = ''
for fn, cid in annots:
    img = cv2.imread(os.path.join(path_dataset, fn))
    if cid == 0:
        r_imgs = augment_image(img, scale_range, asp_range, trans_range)
        for r_img in r_imgs:
            dst = os.path.join(path_res, '{:05d}_{:03d}.jpg'.format(k, cid))
            cv2.imwrite(dst, r_img)
            annot_str += '{:05d}_{:03d}.jpg {}\n'.format(k, cid, cid)
            k += 1

    src = os.path.join(path_dataset, fn)
    dst = os.path.join(path_res, '{:05d}_{:03d}.jpg'.format(k, cid))
    copy(src, dst)
    annot_str += '{:05d}_{:03d}.jpg {}\n'.format(k, cid, cid)
    k += 1

In [49]:
with open(os.path.join(path_res, 'train.txt'), 'w') as fh:
    fh.write(annot_str)